<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# FEC - Création du dataset "Bilan"

**Tags:** #fec #finance #snippet #operation #bilan

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** Ce notebook permet de créer le dataset pour visualiser la décomposition du bilan.

## Input

### Import libraries

In [1]:
import naas_data_product

✅ utils file '/home/ftp/FEC-engine/utils/datalake.ipynb' successfully loaded.


### Setup Variables

In [2]:
# Inputs
input_folder_path = "/home/ftp/FEC-engine/outputs/FEC/bdd_fin_enr"

# Outputs
output_folder_path = "/home/ftp/FEC-engine/outputs/FEC/dataset_bilan"

## Model

### Récupération du dernier fichier input

In [3]:
df_input = get_last_df(input_folder_path)
print("✅ Row fetched:", len(df_input))
df_input.head(1)

📁 Last file: /home/ftp/FEC-engine/outputs/FEC/bdd_fin_enr/20230601154313_FEC_bdd_fin_enr.csv
✅ Row fetched: 296


,ENTITY,PERIOD,COMPTE_NUM,RUBRIQUE_N0,RUBRIQUE_N1,RUBRIQUE_N2,RUBRIQUE_N3,VALUE,VALUE_N-1,VARV,VARP
0,000000000,201712,10130000,PASSIF,PASSIF_NON_COURANT,CAPITAUX_PROPRES,CAPITAL SOUSCRIT APPELÉ VERSÉ,-6000.0,0.0,-6000.0,-inf


### Création du dataset "CHARGES"

In [4]:
# Creation du dataset Bilan
dataset_bilan = df_input.copy()

# Filtre RUBRIQUE_N0 = ACTIF & PASSIF
dataset_bilan = dataset_bilan[(dataset_bilan["RUBRIQUE_N0"].isin(["ACTIF", "PASSIF"]))]

# Regroupement R0/R1/R2
to_group = ["ENTITY", "PERIOD", "RUBRIQUE_N0", "RUBRIQUE_N1", "RUBRIQUE_N2"]
to_agg = {"VALUE": "sum"}
dataset_bilan = dataset_bilan.groupby(to_group, as_index=False).agg(to_agg).fillna(0)


# Mettre en valeur positive VALUE
dataset_bilan["VALUE"] = dataset_bilan["VALUE"].abs()

# Selectionner les colonnes
to_select = ["ENTITY", "PERIOD", "RUBRIQUE_N0", "RUBRIQUE_N1", "RUBRIQUE_N2", "VALUE"]
dataset_bilan = dataset_bilan[to_select]
dataset_bilan["VALUE_D"] = (dataset_bilan["VALUE"] / 1000).map("{:,.1f} k€".format).str.replace(",", " ")

# Affichage du modèle de donnée
dataset_bilan

,ENTITY,PERIOD,RUBRIQUE_N0,RUBRIQUE_N1,RUBRIQUE_N2,VALUE,VALUE_D
0,000000000,201712,ACTIF,ACTIF_COURANT,AUTRES_CREANCES,25148.74,25.1 k€
1,000000000,201712,ACTIF,ACTIF_COURANT,CREANCES_CLIENTS,38959.55,39.0 k€
2,000000000,201712,ACTIF,ACTIF_COURANT,STOCKS,4030.77,4.0 k€
3,000000000,201712,ACTIF,ACTIF_NON_COURANT,IMMOBILISATIONS,79527.73,79.5 k€
4,000000000,201712,ACTIF,DISPONIBILITES,AUTRES DISPONIBILITES,0.00,0.0 k€
5,000000000,201712,ACTIF,DISPONIBILITES,BANQUES & CAISSE,129172.30,129.2 k€
6,000000000,201712,PASSIF,PASSIF_COURANT,AUTRES_DETTES,56471.60,56.5 k€
7,000000000,201712,PASSIF,PASSIF_COURANT,DETTES_FOURNISSEURS,1596.78,1.6 k€
8,000000000,201712,PASSIF,PASSIF_NON_COURANT,CAPITAUX_PROPRES,218770.71,218.8 k€
9,000000000,201812,ACTIF,ACTIF_COURANT,AUTRES_CREANCES,26039.50,26.0 k€


## Output

### Sauvegarde des fichiers en csv

In [5]:
save_df(dataset_bilan, output_folder_path)

✅ DataFrame saved in: /home/ftp/FEC-engine/outputs/FEC/dataset_bilan/20230601155227_FEC_dataset_bilan.csv
